In [1]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
(train_data,train_target),(test_data,test_target) = boston_housing.load_data()

In [21]:
train_data.shape

(404, 13)

In [22]:
test_data.shape

(102, 13)

标准化： $X_t = \frac {X-mean} {std}$

In [12]:
mean = train_data.mean(axis=0) # 这里应该采用train_data + test_data 求均值和标准差
std = train_data.std(axis=0)

In [13]:
train_data = (train_data-mean)/std
test_data = (test_data-mean)/std  #这里操作有点不符合规范

构建神经网络

In [33]:
def build_model():
    model = Sequential()
    model.add(Dense(64,input_shape=(13,),activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(),loss="mse",metrics=['mae'])
    return model

有用数据量比较少，一般采用交叉检验算法

In [25]:
k=4
num_val_samples = len(train_data)//k
num_epochs =100 #100个周期
all_scores =[]

In [41]:
for i in range(k):
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets = train_target[i*num_val_samples:(i+1)*num_val_samples]
    partial_train_data = np.concatenate([train_data[ :i*num_val_samples],
                                       train_data[(i+1)*num_val_samples: ]],axis=0)
    partial_train_target = np.concatenate([train_target[ :i*num_val_samples],
                                       train_target[(i+1)*num_val_samples: ]],axis=0)
    model = build_model()
    model.fit(partial_train_data,partial_train_target, epochs=num_epochs, batch_size=1, verbose=0)#,validation_data=(partial_train_data,partial_train_target))
    
    val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
    all_scores.append(val_mae)
    print("第",i+1,"折，MSE:",val_mse,"MAE:",val_mae)

第 1 折，MSE: 19.61340543539217 MAE: 2.7166715376447925
第 2 折，MSE: 11.638470017083801 MAE: 2.6935975646028423
第 3 折，MSE: 22.290633588734238 MAE: 2.8926395213249885
第 4 折，MSE: 20.038270147720187 MAE: 3.4727178753012478


In [36]:
np.mean(all_scores)

2.7922598203219993

In [37]:
model = build_model()
model.fit(train_data,train_target,epochs=80,batch_size=16,verbose=0)

In [38]:
test_mse,test_mae = model.evaluate(test_data,test_target)

102/102 [==============================] - 0s 725us/step


In [39]:
test_mse

244.20510714661842

In [40]:
np.sqrt(test_mse)

15.627063292462164